In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

from burybarrel.utils import denoise_nav_depth
from burybarrel.image import apply_clahe

In [ ]:
footagecfg_path = Path("../configs/footage.yaml")
footagename = "barrelddt4"

with open(footagecfg_path, "rt") as f:
    footagecfg = yaml.safe_load(f)
navpath = Path("../data/dive-data/Dive8/ROV Dive 8 nav.csv")
cfg = footagecfg[footagename]
vidpath = Path(cfg["path"])
start_t = pd.Timestamp(cfg["start_time"], tz=cfg["timezone"])
outdir = vidpath.parent / vidpath.stem

In [ ]:
crop = True
# top left x, y, width, height
bbox = [0, 120, 1920, 875]
# nav depth data is extremely buggy, values just jump up for no reason
depth_noise_thresh = 6

outdir.mkdir(parents=True, exist_ok=True)
vid = cv2.VideoCapture(str(vidpath))
# videos are at 25 fps
fps = 25
step = 50
dt = pd.Timedelta(seconds=step / fps)
nav = pd.read_csv(navpath)
nav["timestamp"] = pd.to_datetime(nav["timestamp"])
valid_nav = denoise_nav_depth(nav, thresh=depth_noise_thresh, iters=2)
curr_t = start_t
nframes = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
increase_contrast = True

fnames = []
times = []
nav_times = []
lats = []
lons = []
depths = []  # meters
yaws = []  # radians, converted from degree heading measurement relative to north
pitches = []  # radians, converted from being relative to vehicle
rolls = []  # radians
cnt = 0
for i in range(0, nframes, step):
    vid.set(cv2.CAP_PROP_POS_FRAMES, i)
    _, frame = vid.read()
    if frame is None:
        # video ended before expected number of frames
        break
    if increase_contrast:
        frame = apply_clahe(frame, clipLimit=2.0, tileGridSize=(8, 8))
    if crop:
        cropped = frame[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
        fname = f"cropped{str(cnt).zfill(4)}.jpg"
    else:
        cropped = frame
        fname = f"uncropped{str(cnt).zfill(4)}.jpg"
    navrow = valid_nav[valid_nav["timestamp"] <= curr_t].iloc[-1]
    fnames.append(fname)
    times.append(curr_t)
    nav_times.append(navrow["timestamp"])
    lats.append(navrow["latitude"])
    lons.append(navrow["longitude"])
    depths.append(navrow["depth"])
    # convert from degrees to radians
    # change yaw/pitch to be relative to camera, not vehicle
    # yaw is not bearing anymore, so counterclockwise instead
    yaws.append((90 - navrow["heading"]) * np.pi / 180)
    pitches.append((navrow["pitch"] + 90) * np.pi / 180)
    rolls.append(navrow["roll"] * np.pi / 180)
    curr_t += dt
    cv2.imwrite(str(outdir / fname), cropped)
    cnt += 1
df = pd.DataFrame({
    "filename": fnames,
    "timestamp": times,
    "nav_timestamp": nav_times,
    "latitude": lats,
    "longitude": lons,
    "depth": depths,
    "yaw": yaws,
    "pitch": pitches,
    "roll": rolls
})
df.to_csv(outdir / "frame-time-nav.csv", index=False)